# Problem: Implement a CNN for CIFAR-10 (With Custom Layers)

### Problem Statement
You are tasked with implementing a **Convolutional Neural Network (CNN)** for image classification on the **CIFAR-10** dataset using PyTorch. However, instead of using PyTorch's built-in `nn.Conv2d` and `nn.MaxPool2d`, you must implement these layers **from scratch** using `nn.Module`. Your model will include convolutional layers for feature extraction, pooling layers for downsampling, and fully connected layers for classification.

### Requirements
1. **Implement Custom Layers**:
   - Create a custom `Conv2dCustom` class that mimics the behavior of `nn.Conv2d`.
   - Create a custom `MaxPool2dCustom` class that mimics the behavior of `nn.MaxPool2d`.

2. **Define the CNN Model**:
   - Use `Conv2dCustom` for convolutional layers.
   - Use `MaxPool2dCustom` for pooling layers.
   - Use standard `nn.Linear` for fully connected layers.
   - The model should process input images of shape `(3, 32, 32)` as in the CIFAR-10 dataset.

### Constraints
- You must not use `nn.Conv2d` or `nn.MaxPool2d`. Use your own custom implementations.
- The CNN should include multiple convolutional and pooling layers, followed by fully connected layers.
- Ensure the model outputs class predictions for **10 classes**, as required by CIFAR-10.

<details>
  <summary>💡 Hint</summary>
  Define `Conv2dCustom` and `MaxPool2dCustom` as subclasses of `nn.Module`. Use nested loops and tensor slicing to perform the operations.  
  In `CNNModel.__init__`, use these custom layers to build the architecture.  
  Implement the forward pass to pass inputs through convolution, activation, pooling, flattening, and fully connected layers.
</details>


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class Conv2dCustom(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(Conv2dCustom, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride
        self.padding = padding

        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, *self.kernel_size) * 0.1)
        self.bias = nn.Parameter(torch.zeros(out_channels))

    def forward(self, x):
        batch_size, in_channels, H, W = x.shape
        KH, KW = self.kernel_size
        SH = SW = self.stride
        PH = PW = self.padding

        x_padded = F.pad(x, (PW, PW, PH, PH))

        OH = (H + 2 * PH - KH) // SH + 1
        OW = (W + 2 * PW - KW) // SW + 1

        out = torch.zeros((batch_size, self.out_channels, OH, OW), device=x.device)

        for b in range(batch_size):
            for oc in range(self.out_channels):
                for i in range(OH):
                    for j in range(OW):
                        h_start = i * SH
                        h_end = h_start + KH
                        w_start = j * SW
                        w_end = w_start + KW
                        region = x_padded[b, :, h_start:h_end, w_start:w_end]
                        out[b, oc, i, j] = torch.sum(region * self.weight[oc]) + self.bias[oc]
        return out

class MaxPool2dCustom(nn.Module):
    def __init__(self, kernel_size, stride=None):
        super(MaxPool2dCustom, self).__init__()
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if stride is not None else kernel_size

    def forward(self, x):
        batch_size, channels, H, W = x.shape
        KH, KW = self.kernel_size
        SH = SW = self.stride

        OH = (H - KH) // SH + 1
        OW = (W - KW) // SW + 1

        out = torch.zeros((batch_size, channels, OH, OW), device=x.device)

        for b in range(batch_size):
            for c in range(channels):
                for i in range(OH):
                    for j in range(OW):
                        h_start = i * SH
                        h_end = h_start + KH
                        w_start = j * SW
                        w_end = w_start + KW
                        region = x[b, c, h_start:h_end, w_start:w_end]
                        out[b, c, i, j] = torch.max(region)
        return out


In [3]:
# Define the CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Output: 32x32x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: 64x32x32
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 64x16x16
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 1.4057
Epoch [2/10], Loss: 0.9852
Epoch [3/10], Loss: 0.2743
Epoch [4/10], Loss: 0.9068
Epoch [5/10], Loss: 0.2459
Epoch [6/10], Loss: 0.4891
Epoch [7/10], Loss: 0.0719
Epoch [8/10], Loss: 0.1010
Epoch [9/10], Loss: 0.0075
Epoch [10/10], Loss: 0.1189


In [5]:
# Evaluate on the test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 67.59%
